# Vectorstore retrieval

### Similarity Search

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow. 

Given a question, we want to retrieve relevant splits.

Last time, we saw that we can [search for](https://www.pinecone.io/learn/what-is-similarity-search/) semantically relevant docs from our vectordb.

Let's recap that:

In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
vectordb = FAISS.load_local("docs/cs229_lecture1_faiss_index",embd)

In [2]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

### Maximum marginal relevance

Now, let's go a bit beyond this.

What happens in we want to ensure diversity in the resulting chunks?
 
Many vectorstores offer additional search methods to support this.
 
For example, using `maximum marginal relevance` [strives to achieve](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) both relevance to the query *and diversity* among the results.

In [3]:
question = "What are major topics for this class?"
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
len(docs_mmr)

3

### Working with metadata

In addition, many vectorstores support operations on `metadata`.

This provides context for each embedded chunk.

But [we have an interesting challenge](https://twitter.com/hwchase17/status/1651617956881924096?s=20): 

Vector search is good for capturing semantically similar text. 

But, often queries specify desired attributes like time, authorship, or other "metadata" fields.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, [so this doesn't require any new databases or indexes](https://python.langchain.com/docs/modules/data_connection/retrievers/how_to/self_query/chroma_self_query).

Let's try it out with Chroma!

In [ ]:
! pip install lark
! pip install chromadb 

In [6]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

Define our data and create the vectorDB.

In [7]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "rating": 9.9,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, embeddings)

We need to provide some (minimal) information on the fields in the metadata available to filter on.

Now a combination of a query and metadata filter

User Query: "Has Greta Gerwig directed any movies about women"

```
Extracted Query: women
Metadata Filter: director = Greta Gerwig
```

In [9]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

retriever.get_relevant_documents("Has Greta Gerwig directed any movies about women")

query='women' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Greta Gerwig') limit=None


[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3})]

### < Placeholder: Contextual Compression, Time weighting >

# Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` [retriever abstraction](https://blog.langchain.dev/retrieval/) includes other ways to retrieve documents, such as [TF-IDF](https://towardsdatascience.com/tf-term-frequency-idf-inverse-document-frequency-from-scratch-in-python-6c2b61b78558) or [SVMs](https://twitter.com/hwchase17/status/1647328542529843200?s=20). 

In [13]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("docs/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embeddings)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [14]:
question = "What are major topics for this class?"

In [15]:
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes", metadata={})